In [1]:
from datasets import load_dataset

ds = load_dataset("/mnt/workspace/dataset/OpenR1-Math-220k", "default")

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 93733/93733 [00:19<00:00, 4901.19 examples/s]


In [10]:
unused_columns = ["uuid", "is_reasoning_complete", "generations", "correctness_math_verify", "correctness_llama", "finish_reasons", "correctness_count", "messages"]
sub_ds = ds["train"].remove_columns(unused_columns).select(range(1000))
print(sub_ds)
sub_ds_len = sub_ds.map(lambda x: {"length": len(x["problem"])})
print(sub_ds_len)

Dataset({
    features: ['problem', 'solution', 'answer', 'problem_type', 'question_type', 'source'],
    num_rows: 1000
})
Dataset({
    features: ['problem', 'solution', 'answer', 'problem_type', 'question_type', 'source', 'length'],
    num_rows: 1000
})


In [11]:
sub_ds[0]

{'problem': '## Task B-1.3.\n\nA ship traveling along a river has covered $24 \\mathrm{~km}$ upstream and $28 \\mathrm{~km}$ downstream. For this journey, it took half an hour less than for traveling $30 \\mathrm{~km}$ upstream and $21 \\mathrm{~km}$ downstream, or half an hour more than for traveling $15 \\mathrm{~km}$ upstream and $42 \\mathrm{~km}$ downstream, assuming that both the ship and the river move uniformly.\n\nDetermine the speed of the ship in still water and the speed of the river.',
 'solution': '## Solution.\n\nLet $t$ be the time required for the boat to travel $24 \\mathrm{~km}$ upstream and $28 \\mathrm{~km}$ downstream, $v_{R}$ the speed of the river, and $v_{B}$ the speed of the boat. When the boat is traveling upstream, its speed is $v_{B}-v_{R}$, and when it is traveling downstream, its speed is $v_{B}+v_{R}$.\n\nSince $t=\\frac{s}{v}$, from the given data, we obtain the following system of equations:\n\n$\\left\\{\\begin{array}{l}t=\\frac{24}{v_{B}-v_{R}}+\\fra

In [12]:
def generate_conversation(examples):
    problems = examples["problem"]
    solutions = examples["solution"]
    answers = examples["answer"]
    conversations = []
    for problem, solution, answer in zip(problems, solutions, answers):
        conv = [
            {"role": "developer", "content": "You are a good math problem solver."},
            {"role": "user",  "content": problem},
            {"role": "assistant", "content": answer, "thinking": solution}
        ]
        conversations.append(conv)

    return {"conversations": conversations}

In [13]:
from transformers import AutoModel, AutoTokenizer

# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name="/mnt/workspace/model/gpt-oss-20b",
#     max_seq_length=2048,
#     load_in_4bit=True,
#     load_in_8bit=False,
#     full_finetuning=False,
# )
model_path = "/mnt/workspace/model/gpt-oss-20b"
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [14]:
sub_ds_conv = tokenizer.apply_chat_template(
    list(sub_ds.map(generate_conversation, batched=True)["conversations"]), 
    tokenize=False
)

Map: 100%|██████████| 1000/1000 [00:00<00:00, 64717.92 examples/s]


In [15]:
sub_ds_conv[0]

'<|start|>system<|message|>You are ChatGPT, a large language model trained by OpenAI.\nKnowledge cutoff: 2024-06\nCurrent date: 2026-01-27\n\nReasoning: medium\n\n# Valid channels: analysis, commentary, final. Channel must be included for every message.<|end|><|start|>developer<|message|># Instructions\n\nYou are a good math problem solver.\n\n<|end|><|start|>user<|message|>## Task B-1.3.\n\nA ship traveling along a river has covered $24 \\mathrm{~km}$ upstream and $28 \\mathrm{~km}$ downstream. For this journey, it took half an hour less than for traveling $30 \\mathrm{~km}$ upstream and $21 \\mathrm{~km}$ downstream, or half an hour more than for traveling $15 \\mathrm{~km}$ upstream and $42 \\mathrm{~km}$ downstream, assuming that both the ship and the river move uniformly.\n\nDetermine the speed of the ship in still water and the speed of the river.<|end|><|start|>assistant<|channel|>analysis<|message|>## Solution.\n\nLet $t$ be the time required for the boat to travel $24 \\mathrm

In [36]:
from datasets import load_dataset

path = "/mnt/workspace/dataset/mlabonne/FineTome-100k/data"

fine_tome = load_dataset(path)

Generating train split: 100000 examples [00:01, 91838.99 examples/s]


In [38]:
fine_tome = fine_tome["train"]

In [39]:
fine_tome[0]

{'conversations': [{'from': 'human',
   'value': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.'},
  {'from': 'gpt

In [44]:
path = "/mnt/workspace/dataset/OpenMathReasoning-mini"
openr1_mini = load_dataset(path)

In [45]:
openr1_mini = openr1_mini["cot"]

In [46]:
openr1_mini[0]

{'expected_answer': '14',
 'problem_type': 'has_answer_extracted',
 'problem_source': 'aops_c4_high_school_math',
 'generation_model': 'DeepSeek-R1',
 'pass_rate_72b_tir': '0.96875',
 'problem': 'Given $\\sqrt{x^2+165}-\\sqrt{x^2-52}=7$ and $x$ is positive, find all possible values of $x$.',
 'generated_solution': "<think>\nOkay, let's see. I need to solve the equation √(x² + 165) - √(x² - 52) = 7, and find all positive values of x. Hmm, radicals can be tricky, but maybe if I can eliminate the square roots by squaring both sides. Let me try that.\n\nFirst, let me write down the equation again to make sure I have it right:\n\n√(x² + 165) - √(x² - 52) = 7.\n\nOkay, so the idea is to isolate one of the radicals and then square both sides. Let me try moving the second radical to the other side:\n\n√(x² + 165) = 7 + √(x² - 52).\n\nNow, if I square both sides, maybe I can get rid of the square roots. Let's do that:\n\n(√(x² + 165))² = (7 + √(x² - 52))².\n\nSimplifying the left side:\n\nx² + 